# Python Assignment 2

CS16BTECH11038

## Q5

In [159]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [160]:
df_test_raw = pd.read_csv('q4/test.csv')
df_train_raw = pd.read_csv('q4/train.csv')
df_train_raw.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

## Pre-processing

In [162]:
# any pre processing here.

# dealing with missing data!
total = df_train_raw.isnull().sum().sort_values(ascending=False)
percent = (df_train_raw.isnull().sum()/df_train_raw.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Frac'])
missing_data.head(20)

 

,Total,Frac
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


As can be seen, many features are missing in a high fraction of input data (for eg PoolQC, MiscFeature etc). So, I am removing those columns, since they should not make much a difference on the salesprice (maybe that's why they are missing).  

For electrical, only **1** data point has a missing value. Also, electrical intuitively seems to be something that would definitely affect sales price. So, instead of removing that column, I am removing the training data point with the missing electrical value. 

**Note** We already have limited training data (only 1460 samples). So it might be hurtful to remove data points (rows) as a whole, that is why I am dropping features (columns) where missing values are present. 

In [163]:
# Drop all columns except electrical.
df_train_raw = df_train_raw.drop((missing_data[missing_data['Total'] > 1]).index,1)

# Neeed to do this in test set too
df_test = df_test_raw.drop((missing_data[missing_data['Total'] > 1]).index,1)

print(df_train_raw.columns)
print(df_test_raw.columns)


Index(['Id', 'MSSubClass', 'MSZoning', 'LotArea', 'Street', 'LotShape',
       'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'GarageCars', 'GarageArea',
       'PavedDrive', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SaleType',
       'SaleCondition', 'SalePrice'],
      dtype='object')
Index(['Id', 'MSSubClass', 'MSZoni

In [164]:
# One extra column is the salesprice (in training set)
print(df_train_raw.columns.difference( df_test.columns))

Index(['SalePrice'], dtype='object')


In [165]:
# Drop the data point where Electrical value is missing
df_train_raw = df_train_raw.drop(df_train_raw.loc[df_train_raw['Electrical'].isnull()].index)

In [166]:
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor

In [169]:
df_y = df_train_raw.iloc[:,-1]
df_train = df_train_raw.iloc[:,:-1]

# Get dummy variables for non-numeric arguments.
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

missing_cols = df_train.columns.difference(df_test.columns)
for c in missing_cols:
    df_test[c] = 0

# Need to align the columns!
df_train, df_test = df_train.align(df_test, axis=1)

In [170]:
print(df_train.columns.difference( df_test.columns))

Index([], dtype='object')


In [182]:
df_test.isnull().sum().sort_values(ascending=False).head(10)

BsmtHalfBath           2
BsmtFullBath           2
GarageArea             1
BsmtFinSF1             1
BsmtUnfSF              1
TotalBsmtSF            1
BsmtFinSF2             1
GarageCars             1
Exterior2nd_MetalSd    0
Exterior2nd_Other      0
dtype: int64

Still some test data is missing. It is simply filled with 0

In [183]:
df_test = df_test.fillna(0)
df_test.isnull().sum().sort_values(ascending=False).head(10)

YrSold                 0
Exterior2nd_CmentBd    0
Fireplaces             0
Exterior2nd_Wd Shng    0
Exterior2nd_Wd Sdng    0
Exterior2nd_VinylSd    0
Exterior2nd_Stucco     0
Exterior2nd_Stone      0
Exterior2nd_Plywood    0
Exterior2nd_Other      0
dtype: int64

## Evaluating the model using cross validation

In [189]:
reg = MLPRegressor(max_iter=500, hidden_layer_sizes=100)

In [190]:
print(cross_val_score(reg, df_train, df_y, cv=4, n_jobs=4))

[0.7514925  0.72705204 0.71398013 0.49788996]


In [191]:
reg.fit(df_train, df_y)

/home/supreets51/anaconda3/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
             beta_2=0.999, early_stopping=False, epsilon=1e-08,
             hidden_layer_sizes=100, learning_rate='constant',
             learning_rate_init=0.001, max_iter=500, momentum=0.9,
             n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
             random_state=None, shuffle=True, solver='adam', tol=0.0001,
             validation_fraction=0.1, verbose=False, warm_start=False)

In [192]:
test_y = reg.predict(df_test)

In [193]:
# Write the predictions to a CSV file 
submission = pd.DataFrame(
    {'Id': df_test.Id, 'SalePrice': test_y},
    columns = ['Id', 'SalePrice'])
submission.to_csv('submission.csv', index = False)

### First Submission
Hidden layers = 1 (100 neurons per layer), max iterations = 500, activation function = relu, solver = adam

This gave a score (RMSLE) of **0.22938**

### Second Submission

In [205]:
reg = MLPRegressor(max_iter=700, hidden_layer_sizes=(300,310))
#print(cross_val_score(reg, df_train, df_y, cv=4, n_jobs=4))

In [206]:
reg.fit(df_train, df_y)
test_y = reg.predict(df_test)
# Write the predictions to a CSV file 
submission = pd.DataFrame(
    {'Id': df_test.Id, 'SalePrice': test_y},
    columns = ['Id', 'SalePrice'])
submission.to_csv('submission.csv', index = False)

Here, I added one more hidden layer and increased the number of neurons in each layer: 300 and 310. The max number of iterations were increased to 700. Activation function is still relu and the solver is adam.

This gave a score of **0.1707**

### Third submission

In [208]:
reg = MLPRegressor(max_iter=700, hidden_layer_sizes=(300,310), solver='lbfgs')
#print(cross_val_score(reg, df_train, df_y, cv=4, n_jobs=4))
reg.fit(df_train, df_y)
test_y = reg.predict(df_test)
# Write the predictions to a CSV file 
submission = pd.DataFrame(
    {'Id': df_test.Id, 'SalePrice': test_y},
    columns = ['Id', 'SalePrice'])
submission.to_csv('submission.csv', index = False)

Using the lbfgs solver, the score worsened(lower is better) even though this data set was small (~1459 points)
The score was: **0.21363**

### Best Submission

My best score was 0.1707. This was achieved by having two hidden layers of neurons of size 300 and 310. It was slow to converge, so I had to increase the number of iterations from the default value. The solver I used was 'adam' which generally is used for larger datasets but worked for this smaller dataset too. Adding one more hidden layer of neurons did not help, and increased the RMS error to ~0.18